In [1]:
import os
import leafmap
from samgeo import SamGeo, tms_to_geotiff
from datetime import datetime
import json

In [2]:
# Obtendo a data e hora atual
now = datetime.now()
formatted_date = now.strftime("%Y_%m_%d_%H_%M")

def extract_coordinates(geojson_file):
    with open(geojson_file, 'r') as file:
        data = json.load(file)

    point_coords = []
    for feature in data['features']:
        if feature['geometry']['type'] == 'Point':
            point_coords.append(feature['geometry']['coordinates'])

    return point_coords

### Modelo

In [3]:
# vit_b(91M), vit_l(308M), vit_h(636M)

sam = SamGeo(
    model_type="vit_b",
    automatic=False,
    sam_kwargs=None,
)

### Imagem a ser segmentada

In [4]:
# imagem

image = "C:\\Users\\jac\\git\\proc_imagem\\5_estudo_praia_brava\\dados_google_earth\\2023_05.tif"

# pasta saída, arquivo saída

pasta_saida = f'saida/mascaras_por_ponto/{os.path.splitext(os.path.basename(image))[0]}'

os.makedirs(pasta_saida, exist_ok=True)

mascaras_geradas = f'{pasta_saida}/mascaras_{formatted_date}.tif'

In [5]:
sam.set_image(image)

In [6]:
# pontos e detecção

point_coords = extract_coordinates("C:\\Users\\jac\\git\\proc_imagem\\5_estudo_praia_brava\\dados_google_earth\\2023_05.geojson")

print(point_coords)

[[735612.138252, 7018831.894886], [735607.402886, 7018811.860645], [735602.66752, 7018780.170119], [735592.104012, 7018718.97462], [735582.269021, 7018654.865049], [735571.341253, 7018597.312139], [735561.506262, 7018541.944783], [735544.021834, 7018445.051909], [735534.186843, 7018391.141589], [735519.252227, 7018314.282956], [735502.860575, 7018216.297306], [735494.48262, 7018170.76494], [735517.430932, 7018172.221976], [735486.833183, 7018132.882012], [735482.097817, 7018104.834075], [735499.217986, 7018065.858371], [735478.455228, 7018053.473567], [735458.785246, 7017943.831631], [735467.891719, 7017913.233882], [735448.585996, 7017826.540258], [735426.001943, 7017622.91952], [735410.338809, 7017500.164263], [735399.7753, 7017405.456943], [735392.854381, 7017344.625703], [735384.840684, 7017278.330579], [735380.833836, 7017223.69174], [735368.084774, 7017102.757778], [735373.184399, 7017142.097742], [735364.806444, 7017061.59652], [735354.971453, 7016974.174378], [735347.686274, 70

In [9]:
sam.predict(point_coords, point_labels=1, point_crs="EPSG:31982", output=mascaras_geradas)


AttributeError: 'SamGeo' object has no attribute 'predict_batch'

In [8]:
m = leafmap.Map(center=point_coords[0], zoom=19, height="400px")
m.add_basemap("SATELLITE")
m.layers[-1].visible = False

m.add_raster(image, layer_name="Image")

m.add_raster(mascaras_geradas, layer_name="Mask1", nodata=0, cmap="Blues", opacity=1)

m

Map(center=[735612.138252, 7018831.894886], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_i…